**Environment Setup**

In [ ]:
# Install dependenies
!pip -r ./InvestHK/requirements.txt

# You also need to install pytorch with GPU support to accelerate Deep learning, 
# See https://pytorch.org/get-started/locally/
# Example:

!conda install pytorch torchvision torchaudio pytorch-cuda=11.6 -c pytorch -c nvidia

# To use crawler, you need to install Chrome browser & Corresponding Chrome Webdriver
# See https://chromedriver.chromium.org/downloads
# Example: 

!wget https://chromedriver.storage.googleapis.com/108.0.5359.71/chromedriver_linux64.zip



In [ ]:
# Unzip the .zip file and set the executable path to config
# Example
import InvestHK

InvestHK.CFG.CHROME_DRIVER_PATH = 'L:\\ChromeDriver\\108-0-5359-71\\chromedriver.exe'

# Set the log file path

InvestHK.CFG.CRAWLER_LOG_PATH = 'crawler.log'

**InvestHK: Crawler - How to use**

In [ ]:
# Check InvestHK supported websites
InvestHK.CFG.SUPPORTED_WEBSITES

In [ ]:
# 1. Set up a webdriver instance

driver = InvestHK.setup_webdriver(headless=False, pictures=True, scripts=True)

# You will see a browser window pop up

In [ ]:
# 2. Fetch URL from news website

from datetime import datetime, timedelta

# From yesterday to today (Include both)
to_date = datetime.today()
from_date = datetime.today() - timedelta(days=1)

fetch_results = InvestHK.fetch_url(
                                   driver, 
                                   website='bloomberg',
                                   query='Hong Kong',
                                   from_date=from_date,
                                   to_date=to_date
                                   )
fetch_results

In [ ]:
# 3.Take one result and use analyze_html to extract titles & contents

import requests
result = fetch_results[0]
url = result['url']

driver.get(url)

In [ ]:
InvestHK.analyze_html(driver.page_source, website='bloomberg')

In [ ]:
# 4.Train a model to predict investment leads score

bads = '''Freshfields & Mercer on Hong Kong Board Diversity
United Airlines Plans to Resume Hong Kong Flights From January
GDS Is Said to Explore Singapore Listing After Hong Kong, US
China to Start Swap Connect With Hong Kong in Six Months
Why a Primary Listing in Hong Kong Matters for Alibaba, BiliBili
Colliers Rosanna Tang on HK Property Market
Cryptocurrencies are gaining ground in Asia, says Hong Kong-based crypto platform CoinUnited.io
CNBC Transcript: Franklin Tong Fuk-Kay, Chief Executive Officer, Hong Kong Applied Science and Technology Research Institute
Jinmao Investments' Hong Kong trading debut
New economy companies are capturing investor attention in Hong Kong'''.split('\n') 

goods = '''Fintech Giant Lufax Plans Hong Kong Listing to Hedge US Risk
Citadel Adds Office Space for Growing Hong Kong Teams
Hedge Fund Firm North Rock Bucks Trend With Hong Kong Office
Tencent Music Said to Plan Hong Kong Debut as Soon as Next Week
Hong Kong Telco HKBN Said to Draw Interest From Stonepeak, PAG
Truphone debuts in Hong Kong
RTS Expands Operations in Hong Kong with Platform Equinix
Avature Opens New Office in Hong Kong
Fortnum & Mason to open first overseas store in Hong Kong
Hong Kong and Singapore to work together on blockchain project
More consolidation in Asian private banking as OCBC to buy NAB's Singapore and Hong Kong business
Tencent-backed fintech start-up looks to Hong Kong to expand
This Australian brand is expanding in Hong Kong even though the retail scene is struggling'''.split('\n') 

texts = goods + bads
labels = [1] * len(goods) + [0] * len(bads)

metric = InvestHK.train_new_model('Deberta', texts, labels, './testmodel')

In [ ]:
# 5.Check metrics of trained model
metric

In [ ]:
# 6.Load model and predict score

fetched_titles = [r['title'] for r in fetch_results]

test_titles = '''Apple is going to set new office in Hong Kong
This Australian brand is expanding in Hong Kong even though the retail scene is struggling'''.split('\n')

titles = fetched_titles + test_titles
print(titles)

scores = InvestHK.predict_score('testmodel', titles)

In [ ]:
# 7.See the result
for t, s in zip(titles, scores):
    print(f'{s} - {t}')